<a href="https://colab.research.google.com/github/Sina-Akhavi/bitcoin-timeseries-Forecasting/blob/main/cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('./sample_data')

In [ ]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
# ------------------------------------------------
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
# ------------------------------------------------
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPooling1D, Flatten


maindf = pd.read_csv('./BTC-USD.csv')


In [ ]:
print("Null values: ", maindf.isnull().values.sum())
print("NA values: ", maindf.isnull().values.any())

Null values:  0
NA values:  False


In [ ]:
closedf = maindf[['Date', 'Close']]
closedf


,Date,Close
0,2014-09-17,457.334015
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
...,...,...
2708,2022-02-15,44575.203125
2709,2022-02-16,43961.859375
2710,2022-02-17,40538.011719
2711,2022-02-18,40030.976563


In [ ]:
# closedf
# closedf = closedf[closedf['Date'] > '2021-02-19']

closedf = closedf[closedf['Date'] > '2014-09-17']
closedf = closedf[closedf['Date'] < '2020-08-01']
close_stock = closedf.copy()


In [ ]:
close_stock

,Date,Close
1,2014-09-18,424.440002
2,2014-09-19,394.795990
3,2014-09-20,408.903992
4,2014-09-21,398.821014
5,2014-09-22,402.152008
...,...,...
2140,2020-07-27,10990.873047
2141,2020-07-28,10912.823242
2142,2020-07-29,11100.467773
2143,2020-07-30,11111.213867


In [ ]:
fig = px.line(closedf, x=closedf['Date'], y=closedf['Close'], labels={'date': 'Date', 'close': 'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Considered Period to predict Bitcoin Close Price', plot_bgcolor='white',
                  font_size=15, font_color='black')
fig.show()

In [ ]:
del closedf['Date']
scaler = MinMaxScaler()
closedf = scaler.fit_transform(np.array(closedf).reshape(-1, 1))

closedf

array([[0.01275083],
       [0.0112164 ],
       [0.01194666],
       ...,
       [0.56536035],
       [0.56591659],
       [0.57690316]])

In [ ]:
training_size = int(len(closedf) * 0.80)
test_size = len(closedf) - training_size
train_data, test_data = closedf[0: training_size, :], closedf[training_size: len(closedf), :1]

# train_data[:5]
print('train_data shape: ', train_data.shape)
print('test_data shape: ', test_data.shape)

train_data shape:  (1715, 1)
test_data shape:  (429, 1)


In [ ]:
def create_XtrainYtrain(dataset, time_step):

  data_X, data_Y = [], []

  for i in range(len(dataset) - time_step - 1):
    a = dataset[i: (i + time_step), 0]
    data_X.append(a)
    data_Y.append(dataset[(i + time_step), 0])

  return np.array(data_X), np.array(data_Y)



In [ ]:
time_step = 15
X_train, Y_train = create_XtrainYtrain(train_data, time_step)
X_test, Y_test = create_XtrainYtrain(test_data, time_step)

In [ ]:
print('x_train shape:', X_train.shape)
print('y_train shape:', Y_train.shape)
print('x_test shape:', X_test.shape)
print('y_test shape:', Y_test.shape)

x_train shape: (1699, 15)
y_train shape: (1699,)
x_test shape: (413, 15)
y_test shape: (413,)


In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)

X_train.shape:  (1699, 15, 1)
X_test.shape:  (413, 15, 1)


#Model Construction

In [ ]:
from tensorflow.keras import optimizers

adam = optimizers.Adam(learning_rate=0.0005)  # Replace 0.001 with your preferred learning rate

model = Sequential()

# Convolutional layers

model.add(Conv1D(filters=5, kernel_size=2, activation='relu', input_shape=(15,1)))
model.add(Conv1D(filters=5, kernel_size=3, activation='relu'))

# Pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten layer to transition from convolutional layers to dense layers
# model.add(Flatten())

model.add(LSTM(17, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))


model.compile(loss="mean_squared_error", optimizer=adam)
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test),
                    epochs=200, batch_size=32, verbose=1)

Epoch 1/200
54/54 [==============================] - 3s 12ms/step - loss: 0.0498 - val_loss: 0.1508
Epoch 2/200
54/54 [==============================] - 0s 6ms/step - loss: 0.0374 - val_loss: 0.1034
Epoch 3/200
54/54 [==============================] - 0s 6ms/step - loss: 0.0292 - val_loss: 0.0674
Epoch 4/200
54/54 [==============================] - 0s 7ms/step - loss: 0.0232 - val_loss: 0.0360
Epoch 5/200
54/54 [==============================] - 0s 7ms/step - loss: 0.0170 - val_loss: 0.0235
Epoch 6/200
54/54 [==============================] - 0s 8ms/step - loss: 0.0153 - val_loss: 0.0171
Epoch 7/200
54/54 [==============================] - 1s 10ms/step - loss: 0.0146 - val_loss: 0.0116
Epoch 8/200
54/54 [==============================] - 1s 10ms/step - loss: 0.0133 - val_loss: 0.0137
Epoch 9/200
54/54 [==============================] - 1s 10ms/step - loss: 0.0121 - val_loss: 0.0105
Epoch 10/200
54/54 [==============================] - 1s 10ms/step - loss: 0.0108 - val_loss: 0.0112
Epoc

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# do the inverse transformation

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

original_ytrain = scaler.inverse_transform(Y_train.reshape(-1, 1))
original_ytest = scaler.inverse_transform(Y_test.reshape(-1, 1))

13/13 [==============================] - 0s 3ms/step


In [ ]:
print('Train Data RMSE: ', math.sqrt(mean_squared_error(original_ytrain, train_predict)))
print('Train Data MSE: ', mean_squared_error(original_ytrain, train_predict))
print('Train Data MAE: ', mean_absolute_error(original_ytrain, train_predict))
print('----------------------------------------------------------')
print('Test Data RMSE: ', math.sqrt(mean_squared_error(original_ytest, test_predict)))
print('Test Data MSE: ', mean_squared_error(original_ytest, test_predict))
print('Test Data MAE: ', mean_absolute_error(original_ytest, test_predict))

Train Data RMSE:  1054.5643501274312
Train Data MSE:  1112105.9685596914
Train Data MAE:  882.64555854922
----------------------------------------------------------
Test Data RMSE:  845.914780051151
Test Data MSE:  715571.815108987
Test Data MAE:  692.8989009447641


In [ ]:
# shift train predictions for plotting
look_back = time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back: len(train_predict) + look_back, :] = train_predict
print("Train Predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict) + (look_back * 2) + 1: len(closedf)-1, :] = test_predict
print("Test Predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price', 'Train Predicted close price',  'Test Predicted close price'])

plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                       'train_predicted_close': trainPredictPlot.reshape(1, -1)[0].tolist(),
                       'test_predicted_close': testPredictPlot.reshape(1, -1)[0].tolist()})

fig = px.line(plotdf, x=plotdf['date'], y=[plotdf['original_close'], plotdf['train_predicted_close'], plotdf['test_predicted_close']],
              labels={'value': 'stock price', 'date': 'Data'})

fig.update_layout(title_text='Comparison between original close price vs predicted close price', plot_bgcolor='white', font_size=15,
                  font_color='black', legend_title_text='Close price')

fig.for_each_trace(lambda t: t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Train Predicted data:  (2144, 1)
Test Predicted data:  (2144, 1)
